In [2]:
import sys
sys.setrecursionlimit(1000)
from bs4 import BeautifulSoup
import re 
from collections import defaultdict 
from pandas import Series, DataFrame 
import numpy as np 
import re 
import sys
import unidecode
import glob, os
import csv
import urllib2
import unicodedata
import os
import requests
import lxml
import html5lib
import pandas as pd

In [2]:
def mpaa_rating_adjustment(list_of_variables, mpaaRating): 
    list_to_append = []
    for category in list_of_variables: 
        try: 
            if category == 'Rotten Tomatoes':  
                rating_index = mpaaRating.index(category)+3
                rating = mpaaRating[rating_index:rating_index+2]
                
                list_to_append.extend([rating])
                
            else:
                category = (mpaaRating[mpaaRating.index(category)+1])
                list_to_append.append(category)
                
        except (ValueError, AttributeError):
            
            list_to_append.append(None)
        
    
            
    return list_to_append

response = urllib2.urlopen('http://www.the-numbers.com/movie/budgets/all')
main_doc = response.read()

In [3]:
def txt_link_downloader(html_link, limit):
    
    soup = BeautifulSoup(html_link, 'html5lib')
    list_df = []    
    batch = soup.find_all('td')
    counter = 0
    length = len(batch)
    for index,i in enumerate(range(0,length,6)):
        list_df.append(map(lambda x: x.get_text(), batch[i:i+6]))
        try:
            url_end = BeautifulSoup(batch[i+2].encode('utf-8'),'html5lib').find('a').get('href') 
            url = 'http://www.the-numbers.com' + url_end
            list_df[index].append(url)
            
            response = urllib2.urlopen(url)
            main_doc = response.read()
            soup = BeautifulSoup(main_doc,'html5lib')
        
            mpaaRating = []
            for tr in soup.findAll('tr'): 
                for td in tr.findAll('td'): 
                    mpaaRating.append(td.get_text())
            mpaaRating = [unidecode.unidecode(x).strip() for x in mpaaRating]   
            
            list_of_variables = ['Genre:','Running Time:','MPAA Rating:','Production Companies:','Domestic Releases:','Domestic DVD Sales','Domestic Blu-ray Sales','Total Domestic Video Sales','Rotten Tomatoes']
            
            second_page = mpaa_rating_adjustment(list_of_variables,mpaaRating)
            
            list_df[index].extend(second_page)
            
            response = urllib2.urlopen(url)
            main_doc = response.read()
            soup = BeautifulSoup(main_doc,'html5lib')
            soup = soup.find(text = re.compile('Weekend Box Office Performance')).parent.parent.find('div', attrs = {"id": "box_office_chart"})
            try:
                soup = soup.get_text()
                soup = unicodedata.normalize('NFKD', soup).encode('utf-8').split()[4:35]
                soup.insert(3,'None')
                list_df[index].extend(soup)
            except:
                print('doesnt work 1')
        
            counter += 1
            #print('passing first complete iteration', counter)
            print(counter)
            if counter == limit:
                df = DataFrame(list_df)
                df.columns = range(df.shape[1])
                columns_to_delete = [0,1,18,25,26,33,34,41,42,48]
                for column in columns_to_delete:
                    del df[column]
                    
                columns=['Release Date','Title','Production Budget','Domestic Gross','Worldwide Gross','Url','Genre','Runtime','MPAA','Production Companies','Release','Domestic DVD Sales','Domestic Blu-ray Sales','Total Domestic Video Sales','Critic Rating','Wknd 1 Rank','Wknd 1 Gross','Wknd 1 %Change','Wknd 1 # Theaters','Wknd 1 GPT','Week 1 TG', 'Wknd 2 Rank','Wknd 2 Gross','Wknd 2 %Change','Wknd 2 # Theaters','Wknd 2 GPT','Week 2 TG','Wknd 3 Rank','Wknd 3 Gross','Wknd 3 %Change','Wknd 3 # Theaters','Wknd 3 GPT','Week 3 TG','Wknd 4 Rank','Wknd 4 Gross','Wknd 4 %Change','Wknd 4 # Theaters','Wknd 4 GPT','Week 4 TG']
                df.columns = columns
                df.to_csv('the_numbers_raw.csv',index=None, encoding='utf-8')
        except:
            print('doesnt work 2')

txt_link_downloader(main_doc, 500)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
doesnt work 1
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
doesnt work 1
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
